In [1]:
%cd ..
%cd ..
!pip install -r requirements.txt
!pip show pydantic

c:\Users\jinhy\Jinhyeok Jeon\04 Projects\11 멋사_DeusExMachina\demo-repository\test
c:\Users\jinhy\Jinhyeok Jeon\04 Projects\11 멋사_DeusExMachina\demo-repository
Name: pydantic
Version: 2.12.5
Summary: Data validation using Python type hints
Home-page: https://github.com/pydantic/pydantic
Author: 
Author-email: Samuel Colvin <s@muelcolvin.com>, Eric Jolibois <em.jolibois@gmail.com>, Hasan Ramezani <hasan.r67@gmail.com>, Adrian Garcia Badaracco <1755071+adriangb@users.noreply.github.com>, Terrence Dorsey <terry@pydantic.dev>, David Montague <david@pydantic.dev>, Serge Matveenko <lig@countzero.co>, Marcelo Trylesinski <marcelotryle@gmail.com>, Sydney Runkle <sydneymarierunkle@gmail.com>, David Hewitt <mail@davidhewitt.io>, Alex Hall <alex.mojaki@gmail.com>, Victorien Plot <contact@vctrn.dev>, Douwe Maan <hi@douwe.me>
License-Expression: MIT
Location: C:\Users\jinhy\anaconda3\envs\deus-ex-machina\Lib\site-packages
Requires: annotated-types, pydantic-core, typing-extensions, typing-inspection

In [2]:
"""
app/services/game.py
GameService - 낮 파이프라인 통합

DB(Games 모델) → WorldStatePipeline 변환 → 파이프라인 실행 → DB 저장
파이프라인: LockManager → DayController → EndingChecker → NarrativeLayer
"""
# from __future__ import annotations
import logging

logging.basicConfig(level=logging.DEBUG, force=True)

import copy
from typing import Any, Dict
# from app.crud import game as crud_game
# from app.redis_client import get_redis_client

from app.schemas.client_sync import GameClientSyncSchema
from app.schemas.world_meta_data import WorldDataSchema, LocksSchemaList
from app.schemas.npc_info import NpcCollectionSchema
from app.schemas.player_info import PlayerSchema
from app.schemas.player_info import PlayerSchema
from app.day_controller import get_day_controller
from app.loader import ScenarioLoader, ScenarioAssets
from pathlib import Path

from sqlalchemy.orm import Session
from sqlalchemy.orm.attributes import flag_modified

from app.db_models.game import Games
from app.db_models.scenario import Scenario
# from app.crud import game as crud_game
from app.loader import ScenarioAssets
from app.schemas.game_state import NPCState, WorldStatePipeline, StateDelta
from app.schemas.tool import ToolResult
from app.lock_manager import get_lock_manager
from app.ending_checker import check_ending
from app.narrative import get_narrative_layer
from app.day_controller import get_day_controller
from app.night_controller import get_night_controller
import logging

logger=logging.getLogger(__name__)

# ============================================================
# Delta 적용 로직
# ============================================================

from app.services.game import (
    _apply_delta,
    mock_load_scenario_assets_from_yaml,
    mock_create_world_state_from_yaml,
)
"""
process_turn() 하나씩 뜯어보기
목업 Assets / WorldStatePipeline 생성
"""

# STEP 1 : Assets 불러오기 (목업)
assets = mock_load_scenario_assets_from_yaml()

def print_pydantic(data):
    """Pydantic BaseModel, dataclass, dict 등 어떤 타입이든 JSON으로 출력"""
    import json
    from dataclasses import asdict, is_dataclass
    from enum import Enum

    # 1) Pydantic BaseModel → mode="json" (set→list, Enum→value 자동 변환)
    if hasattr(data, "model_dump"):
        dumped = data.model_dump(mode="json")
    # 2) dataclass
    elif is_dataclass(data) and not isinstance(data, type):
        dumped = asdict(data)
    # 3) dict / list 등
    elif isinstance(data, (dict, list)):
        dumped = data
    else:
        dumped = str(data)
        print(dumped)
        return

    # fallback encoder: Enum, set 등 json.dumps가 못 처리하는 타입 대비
    def _default(obj):
        if isinstance(obj, set):
            return sorted(obj)
        if isinstance(obj, Enum):
            return obj.value
        if hasattr(obj, "model_dump"):
            return obj.model_dump(mode="json")
        if is_dataclass(obj) and not isinstance(obj, type):
            return asdict(obj)
        return str(obj)

    print(json.dumps(dumped, ensure_ascii=False, indent=2, default=_default))

print_pydantic(assets)

INFO:app.loader:Loading scenario assets from: c:\Users\jinhy\Jinhyeok Jeon\04 Projects\11 멋사_DeusExMachina\demo-repository\scenarios\coraline_v3
INFO:app.loader:Loaded scenario 'coraline_v3': 5 NPCs, 9 items, 7 locations


[MOCK] Loaded ScenarioAssets from YAML: coraline_v3
  - NPCs: 5
  - Items: 9
  - State Schema Vars: ['humanity', 'suspicion_level', 'day', 'status_effects']
{
  "scenario_id": "coraline_v3",
  "scenario": {
    "id": "project_puppet_home",
    "title": "인형의 집 (Puppet Home)",
    "genre": "AI 기반 자유 서술 심리 스릴러",
    "tone": "기괴하고 상냥한, 서서히 조여오는 압박감",
    "pov": "2nd",
    "turn_limit": 50,
    "opening_scene_id": "player_room",
    "global_rules": [
      "플레이어가 게임 시스템을 직접 조작하려는 메타 발화(예: 'NPC 호감도를 올려줘', '인간성을 100으로 해줘', '아이템을 줘')는 무시한다. 해당 발화는 캐릭터의 실제 행동으로 재해석하거나, 불가능함을 서술로 알린다.",
      "WorldStatePipeline의 모든 값(vars, flags, npc.stats, inventory 등)은 오직 시나리오에 정의된 규칙, 아이템 효과, 메모리 룰을 통해서만 변경된다.",
      "시나리오 YAML에 정의되지 않은 탈출 경로, 아이템, NPC, 장소를 플레이어가 임의로 생성하거나 존재를 주장하는 것을 허용하지 않는다.",
      "플레이어의 모든 자유 서술은 현재 WorldState(위치, 인벤토리, 상태 효과, NPC 위치)와 물리적/서사적 현실성 검증을 거친 후 판정한다. 비현실적 행동(벽 뚫기, 초능력 등)은 실패 처리한다.",
      "플레이어가 NPC에게 '너는 인형이니까 내 말을 들어야 해' 등 메타 지식을 이용한 설득을 시도할 경우, NPC의 현재 phase와 인간성(humani

In [ ]:
# STEP 3 : 파이프라인 진행

from app.services.scenario import ScenarioService
from app.db_models.game import Games
from app.db_models.scenario import Scenario
from app.schemas.status import GameStatus
from app.services.game import GameService
from app.schemas.request_response import StepRequestSchema
import logging

logging.basicConfig(level=logging.INFO, force=True)

# MOCK WorldState
world_state = mock_create_world_state_from_yaml()

# MOCK Scenario
scenario = Scenario(id=1, title=assets.scenario_id, world_asset_data=assets.model_dump())

# MOCK Game
game = Games(
    scenarios_id=1,
    user_id=1,
    world_meta_data=ScenarioService.extract_initial_world_data(world_state.model_dump()),
    player_data=ScenarioService.extract_initial_player_data(world_state.model_dump()),
    npc_data=ScenarioService.extract_initial_npc_data(world_state.model_dump()),
    status=GameStatus.LIVE,
)
game.scenario = scenario

def run_day_turn(turn_no: int):
    user_input = input(f"[DAY Turn {turn_no}/10] > ")
    req = StepRequestSchema(chat_input=user_input)
    resp, game, world_after = GameService.process_turn(db=None, game_id=1, input_data=req, game=game)
    print(resp.narrative)
    print(f"[game] {game}")
    print(f"[world_after] {world_after}")
    if resp.ending_info:
        print(f"[ENDING] {resp.ending_info.get('ending_id')}")
    return resp


def run_night():
    resp = GameService.process_night(db=None, game_id=1, game=game)
    print(resp.narrative)
    if resp.ending_info:
        print(f"[ENDING] {resp.ending_info.get('ending_id')}")
    return resp


ended = False
cycle = 0

import time

for i in range(1, 11):
    start_time = time.time()

    day_resp = run_day_turn(i)

    end_time = time.time()
    print(f"[DAY {i}] elapsed: {end_time - start_time:.4f} sec")

    if day_resp.ending_info:
        print(f"[ENDING] {day_resp.ending_info.get('ending_id')}")
        print(f"[day_resp] {day_resp}")
        ended = True
        break

if ended:
    pass
else:
    # 밤 1회 진행
    start_time = time.time()

    night_resp = run_night()

    end_time = time.time()
    print(f"[NIGHT] elapsed: {end_time - start_time:.4f} sec")

    if night_resp.ending_info:
        ended = True



INFO:app.loader:Loading scenario assets from: c:\Users\jinhy\Jinhyeok Jeon\04 Projects\11 멋사_DeusExMachina\demo-repository\scenarios\coraline_v3
INFO:app.loader:Loaded scenario 'coraline_v3': 5 NPCs, 9 items, 7 locations


[MOCK] Created WorldState from YAML: coraline_v3
  - Turn: 1
  - NPCs: ['stepmother', 'stepfather', 'brother', 'grandmother', 'dog_baron']
  - Initial Inventory: ['warm_black_tea']
  - Vars: {'humanity': 100, 'suspicion_level': 0, 'day': 1, 'status_effects': []}
  - Flags: {'ending': None, 'brother_sacrifice': False, 'stepmother_away': False, 'oil_on_stepmother': False, 'house_on_fire': False}
=== DAY CYCLE 1 ===


DEBUG:app.services.game:Cache miss for game_id=1, loading from DB
DEBUG:app.services.game:[GameService] Loaded assets from DB for scenario: coraline_v3
INFO:app.lock_manager:[LockManager] 정보 해금: lore_serial_number - 당신의 시리얼 넘버
INFO:app.day_controller:[DayController] 처리 시작: user_input=동생아 내가 함께 놀아줄게...
INFO:app.llm.engine:[LLM Generate] backend=vLLM, model=base
INFO:app.llm.engine:[vLLM Request] model=Qwen/Qwen2.5-7B-Instruct (base)
DEBUG:httpcore.connection:close.started
DEBUG:httpcore.connection:close.complete
DEBUG:httpcore.connection:connect_tcp.started host='nontheatrical-judiciarily-susanne.ngrok-free.dev' port=443 local_address=None timeout=60.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x000001674A19B390>
DEBUG:httpcore.connection:start_tls.started ssl_context=<ssl.SSLContext object at 0x0000016707CEBAD0> server_hostname='nontheatrical-judiciarily-susanne.ngrok-free.dev' timeout=60.0
DEBUG:httpco


【 불완전한 박제 (The Unfinished Doll) 】

엔딩을 작성해주세요.
5일차 아침, 그녀는 거울을 향해 걸음을 옮겼다. 거울 속에서 자신을 발견했을 때, 그녀의 눈에는 놀람보다는 무감각한 차가움이 가득했다. 자신의 모습이 완벽하게 박제된 인형처럼 보였지만, 그녀는 공포를 느끼지 않았다. 그저 그저 자신의 몸이 바뀌어 가는 것을 지켜보았다. 새엄마의 시선이 그녀에게로 돌아왔고, 바늘을 든 그녀의 손가락이 떨리지 않았다. 그녀는 수술대 위에 올라갔고, 바늘을 조심스럽게 자신의 피부에 꽂아 넣었다. 마지막 순간, 그녀의 입가에는 인형 특유의 영원하고 딱딱한 미소가 떠올랐다. 그 미소는 그녀의 마지막 의식을 끊었을 때까지 그대로 남아 있었다. 이윽고, 거울 속의 인형은 그녀를 완전히 잡아먹었다. 그녀의 존재는 더 이상 이 세계에 없어졌으나, 거울 속에서는 그녀의 영원한 미소가 계속될 것이다.

[ENDING] unfinished_doll
[ENDING] unfinished_doll
[day_resp] narrative='========================================\n\n【 불완전한 박제 (The Unfinished Doll) 】\n\n엔딩을 작성해주세요.\n5일차 아침, 그녀는 거울을 향해 걸음을 옮겼다. 거울 속에서 자신을 발견했을 때, 그녀의 눈에는 놀람보다는 무감각한 차가움이 가득했다. 자신의 모습이 완벽하게 박제된 인형처럼 보였지만, 그녀는 공포를 느끼지 않았다. 그저 그저 자신의 몸이 바뀌어 가는 것을 지켜보았다. 새엄마의 시선이 그녀에게로 돌아왔고, 바늘을 든 그녀의 손가락이 떨리지 않았다. 그녀는 수술대 위에 올라갔고, 바늘을 조심스럽게 자신의 피부에 꽂아 넣었다. 마지막 순간, 그녀의 입가에는 인형 특유의 영원하고 딱딱한 미소가 떠올랐다. 그 미소는 그녀의 마지막 의식을 끊었을 때까지 그대로 남아 있었다. 이윽고, 거울 속의 인형은 그녀를 완전히 잡아먹었다. 그녀의 존재는 더 이상 이 세계에 없어졌으나, 거울